In [6]:
from contrans import contrans
import numpy as np
import pandas as pd
import requests
import dotenv
import os
import json
import requests
import psycopg
import sqlite3
from sqlalchemy import create_engine
from bs4 import BeautifulSoup


dotenv.load_dotenv()
congresskey= os.getenv("congresskey")
postgrespassword= os.getenv("POSTGRES_PASSWORD")

SyntaxError: invalid syntax (contrans.py, line 273)

In [ ]:
ct = contrans()
ct.mypassword
ct.POSTGRES_PASSWORD

In [ ]:
useragent = ct.get_useragent()
useragent


In [ ]:
dbserver, engine = ct.connect_to_postgres(ct.POSTGRES_PASSWORD, create_contrans = True)

In [ ]:
members = ct.get_bioguideIDs()
members = ct.make_cand_table()
terms, members = ct.terms_df(members)
ideology = ct.get_ideology()
ct.make_members_df(members, ideology, engine)

In [ ]:
ct.make_terms_df(terms,engine)

In [ ]:
votes=ct.get_votes()
ct.make_votes_df(votes, engine)

In [ ]:
myquery = '''
SELECT 
bioguideid FROM members
'''
pd.read_sql_query(myquery, con = engine)

In [ ]:
myquery = '''
SELECT * FROM members
'''
data = pd.read_sql_query(myquery, con = engine)
print(ct.dbml_helper(data))

In [ ]:
myquery = '''
SELECT * FROM votes
'''
data = pd.read_sql_query(myquery, con = engine)
print(ct.dbml_helper(data))

In [ ]:
myquery = '''

SELECT * FROM members 
WHERE state = 'VA'
'''
pd.read_sql_query(myquery, con =engine)


In [ ]:
myquery = '''

SELECT 
    name AS congressperson,
    partyname AS party,
    state   AS us_state,
    district AS congressional_district
FROM members 
WHERE partyname = 'Democratic'
ORDER BY congressperson DESC
LIMIT 10
'''
pd.read_sql_query(myquery, con =engine)


In [ ]:
# create voting affinity table
myquery = '''
with member_14854 as (
SELECT 
    a.icpsr AS icpsr1,
    b.icpsr AS icpsr2,
    AVG(CASE WHEN (a.cast_code = b.cast_code) THEN 1 ELSE 0 END) AS agree
FROM votes a
INNER JOIN votes b 
    ON a.chamber = b.chamber
    AND a.rollnumber = b.rollnumber
WHERE a.icpsr = 14854 AND b.icpsr != 14854
GROUP BY icpsr2, icpsr1
ORDER BY agree DESC
), members_info as (
SELECT
    name,
    partyname,
    state,
    district,
    CAST(icpsr AS INT) AS icpsr
FROM members)
SELECT 
    m.name,
    m.partyname,
    m.state,
    m.district,
    m.icpsr,
    v.agree
FROM member_14854 v
INNER JOIN members_info m
    ON v.icpsr2 = m.icpsr
WHERE m.icpsr IS NOT NULL
ORDER BY agree DESC

'''
pd.read_sql_query(myquery, con =engine)

In [ ]:
import pymongo

MONGO_INITDB_ROOT_USERNAME=os.getenv("MONGO_INITDB_ROOT_USERNAME")
MONGO_INITDB_ROOT_PASSWORD=os.getenv("MONGO_INITDB_ROOT_PASSWORD")
 

In [ ]:
myclient.list_database_names()

#myclient is the Mongo server
mongo_contrans=myclient['contrans']
#create a database
mongo_bills = mongo_contrans['bills']

In [ ]:
# Create JSON/Dictionary records to store in the collection
myquery = '''
SELECT 
bioguideId
FROM members
'''
bioguides =pd.read_sql_query(myquery, con =engine)['bioguideid'].tolist()

In [ ]:
headers=ct.make_headers()
headers

In [ ]:
bill_list = ct.get_sponsoredlegislation(bioguides[0])
#onebill = ct.get_billdata(bill_list[0]['url']) 
allbills = [ct.get_billdata(x['url']) for x in bill_list]
allbills

In [ ]:
mongo_bills.insert_many(allbills)

In [ ]:
myclient.list_database_names()

In [ ]:
ideology = ct.get_ideology()
ideology

In [ ]:
#indcontrib = pd.read_csv('data/CampaignFin22/indivs22.txt', quotechar="|", header=None)
#indcontrib

In [ ]:
members['terms.item']


In [ ]:
terms,members = ct.terms_df(members)

In [ ]:
members

In [ ]:
replace_map = {'Republican': 'R',
                'Democratic': 'D', 
                'Independent': 'I'}

members['partyletter']= members['partyName'].replace(replace_map)
members

In [ ]:
members['lastname'] = [x.split(',')[0] for x in members['name']]
members['firstname'] = [x.split(',')[1] for x in members['name']]
members['name2']= [x +' '+y + ' (' +z + ')' for x,y,z in 
                   zip(members['firstname'], members['lastname'], members['partyletter'])]
members

In [ ]:
crosswalk = pd.merge(members, cands, left_on='name2',
                     right_on='FirstLastP', 
                     how='outer', 
                     indicator= 'matched',
                     validate='one_to_one')

crosswalk

In [ ]:
members

In [ ]:
# What are we matching on?

# What kind of merge is this? One to one?

#

In [ ]:
print(ct.get_bioguideIDs)


In [ ]:
goodbioguide = ct.get_bioguide(name = 'good', state = 'Virginia', district = 5)
goodbioguide

In [ ]:
goodbioguide['bioguideId'].reset_index()

In [ ]:
goodid = goodbioguide['bioguideId'].reset_index()['bioguideId'][0]

In [ ]:
goodid

In [ ]:
#Sponsored legislation
goodsbills = ct.get_sponsoredlegislation(goodid)


In [ ]:
r = requests.get(goodsbills[0]['url'],
                 params = {'api_key':congresskey}) #goodsbills[0]['url']

texturl = json.loads(r.text)['bill']['textVersions']['url']


r = requests.get(texturl,
                 params= {'api_key':congresskey})
json.loads(r.text)['textVersions'][0]['formats'][0]['url']

toscrape = json.loads(r.text)['textVersions'][0]['formats'][0]['url']

In [ ]:
from bs4 import BeautifulSoup

r = requests.get(toscrape)
mysoup = BeautifulSoup(r.text, 'html.parser') #equivalent of json.loads, allows you to search through a string
billtext = mysoup.text

In [ ]:
billurl = goodsbills[0]['url']

In [ ]:
ct.get_billdata(billurl)

In [ ]:
print(mysoup.text)

In [ ]:
votes = ct.get_votes()
votes

In [ ]:
house = house(['rollnumber', 'icpsr','cast_code'])

In [ ]:
house_mat = pd.merge(house, house on = 'rollnumber')
house_mat

In [ ]:
house_mat['agree'] = house_mat['cast_code_x'] == house_mat['cast_code_y']
house_mat = house_mat.groupby(['icpsr_x', 'icpsr_y']).agg({'agree': 'mean'})
house_mat = house_mat.reset_index()
house_mat

In [ ]:
ideo = ct.get_ideology()
members = ideo[['bioname', 'icpsr','partycode']]
members

In [ ]:
agreement = pd.merge(house_mat, members, 
                     left_on = 'icpsr_x', 
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member1',
                              'party_code':'party1'}, axis=1)
agreement = agreement.drop(['icspr_x'],axis=1)
agreement = pd.merge(agreement, members,
                     left_on = 'icspr_y',
                     right_on = 'icpsr')
agreement = agreement.rename({'bioname':'member2',
                              'party_code':'party2'}, axis=1)
agreement = agreement[['member1','party1','member2','party2','agree']]
agreement

In [ ]:
agreement[agreement['member1'].str.contains('GOOD,')].sort_values('agree', ascending = False).head(30)